In [8]:
# reading MODIS daily files, running through a year,
#  and outputting a mean grid at its native resolution as downloaded
# updated dduncan 19/1/18 to allow selection of aqua-only vs. combined aqua/terra grids
# updated dduncan 30/1/18 for weighting by cloud retrieval fraction (that's what others in
#  the literature do, documentation is unclear!), don't have terra data though so it's aqua-only rn
from pyhdf.SD import SD, SDC
import glob

In [9]:
mdir = '/home/dudavid/Dendrite/Dendrite/UserAreas/Dave/MODIS/'
yr = '15' # year 20x
f_terra = sorted(glob.glob(mdir+'MOD08_D3.A20'+yr+'*.hdf'))
f_aqua  = sorted(glob.glob(mdir+'MYD08_D3.A20'+yr+'*.hdf'))
aqua = 1 # if on, use aqua data only, else it's a blended terra/aqua grid that's output

In [10]:
gsize = 1.0 # in degrees, normal grid
latmax= 90  # NS latitude limits of grid (limited due to daylight )
yy = 90-latmax
nx, ny = int(360.0/gsize), int(2.0*latmax/gsize) 
mean_grid_iwp = np.zeros([ny, nx])
zonal_mean_iwp = np.zeros(ny)
count = np.zeros([ny, nx]) # initialize
ndays = 0
iwpall =[]
print(ny,nx)
print(len(f_terra),len(f_aqua))

180 360
365 365


In [11]:
for d in range(len(f_aqua)):#[0:1]:
#for d in range(len(f_terra[0:30])):
    if ndays % 20 == 0: print(d)
    if aqua == 0: 
        t = SD(f_terra[d],SDC.READ)  # tried looking at maxima just once!
        te_iwp =  0.001 * t.select('Cloud_Water_Path_Ice_Mean')[yy:(180-yy),:]
        te_crf = 0.0001 * t.select('Cloud_Retrieval_Fraction_Ice')[yy:(180-yy),:] # NEW 30/1!
        #te_iwp =  0.001 * t.select('Cloud_Water_Path_Ice_Maximum')[yy:(180-yy),:]
    a = SD(f_aqua[d],SDC.READ) 
    aq_iwp =  0.001 * a.select('Cloud_Water_Path_Ice_Mean')[yy:(180-yy),:]
    aq_crf = 0.0001 * a.select('Cloud_Retrieval_Fraction_Ice')[yy:(180-yy),:] # NEW 30/1!
    #print(info(aq_crf))
    #aq_iwp =  0.001 * a.select('Cloud_Water_Path_Ice_Maximum')[yy:(180-yy),:]
    if aqua==1: 
        te_iwp = aq_iwp*0 - 9.999 # if wanting aqua-only IWP output
    aq_iwp[ aq_iwp < 0 ] = np.nan
    te_iwp[ te_iwp < 0 ] = np.nan
    ndays += 1 # count days processed
    modis_daily_combined = np.zeros([ny,nx]) #+ np.nan
    dcount = np.zeros([ny,nx])
    #print(np.nanmax(aq_iwp),np.nanmax(te_iwp))
    for x in range(nx):
        for y in range(ny):
            if (aq_iwp[y,x]>=0 and te_iwp[y,x]>=0): \
                modis_daily_combined[y,x] = (aq_iwp[y,x]*aq_crf[y,x] + te_iwp[y,x]*te_crf[y,x])*.5
            if (aq_iwp[y,x]>=0 or  te_iwp[y,x]>=0): dcount[y,x] = 1
            if (aq_iwp[y,x]>=0 or  te_iwp[y,x]>=0): count[y,x] += 1
            if (aq_iwp[y,x]>=0 and np.isnan(te_iwp[y,x])): \
                modis_daily_combined[y,x] = aq_iwp[y,x]*aq_crf[y,x]
            if (np.isnan(aq_iwp[y,x]) and te_iwp[y,x]>=0): \
                modis_daily_combined[y,x] = te_iwp[y,x]*te_crf[y,x]
            #if (aq_iwp[y,x]<0 and te_iwp[y,x]>=0): modis_daily_combined[y,x] = te_iwp[y,x]
    mean_grid_iwp[dcount > 0] += modis_daily_combined[dcount > 0] #really a total of good points
    #print(np.mean(modis_daily_combined[ dcount > 0]))
    zonal_mean_iwp[:] += np.nanmean(modis_daily_combined[:,:],axis=1)
    #print(zonal_mean_iwp)
    #totice = np.reshape(modis_daily_combined[:,:], [ny,nx])
    #iwpall.extend(modis_daily_combined[ dcount > 0 ])

print(info(mean_grid_iwp))
print(info(count))
mean_grid_iwp[count > 0] /= count[count > 0]  #divide by array of counts now
zonal_mean_iwp /= float(ndays)
print(info(mean_grid_iwp))
print(info(zonal_mean_iwp))

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
[0.0085460000000000015, 90.568087300000016, 14.033056501050927, 9.8719424263611284]
[9.0, 345.0, 169.32256172839507, 60.197549760278875]
[0.00077690909090909102, 0.34206659108527127, 0.074040057459642233, 0.03612518717842287]
[0.0067195436750380521, 0.077942953552511365, 0.038446730139865555, 0.018512965102152289]


In [12]:
#iwpall = iwpall[ iwpall == iwpall] #get rid of one nan value?
#print(iwpall[iwpall != iwpall])
# create histogram of IWP
#binz = [2**x for x in range(-14,5)]
#hista, bin_edges  = np.histogram(np.array(iwpall),bins=binz)

In [13]:
nd = str(ndays)
# dataset name, res (x,y), time res of input, output arrays (grids, zonals), version?, # days
#out = open('grids/modis.'+str(nx)+'x.'+str(ny)+'y.daily.iwp.zi.hist.v1.'+nd+'.bin','wb') #write binary mode
if aqua:  # v2 for using cloud retrieval fraction to weight IWP values
    fin = 'grids/modis-aqua.'+str(nx)+'x.'+str(ny)+'y.daily.iwp.zi.hist.v2.'+yr+'.'+nd
else:
    fin = 'grids/modis-combined.'+str(nx)+'x.'+str(ny)+'y.daily.iwp.zi.hist.v2.'+yr+'.'+nd
np.save(fin+'.grid',mean_grid_iwp)
np.save(fin+'.zonal',zonal_mean_iwp)
#out.write(mean_grid_iwp)
#out.write(mean_grid_swp)
#out.write(zonal_mean_iwp)
#out.write(zonal_mean_swp)
#out.write(hista) # counts from given bins (2e-14 to 2e5)
#out.close()

In [ ]:
#fig = plt.figure()
#ax = fig.add_subplot(111)
#plt.plot(binz[1:],hista/sum(hista),'r-o',label='SWP+CIWP')
#plt.plot(binz[1:],histi/sum(histi),'b-o',label='Cloud ice')
#plt.plot(binz[1:],hists/sum(hists),'k-o',label='Snow')
#plt.xscale('log')
#plt.xlabel(r'$IWP [kg/m^2]$')
#plt.ylabel('Fraction of cloudy counts')
#plt.title('Oceanic IWP and SWP, '+mon+' ERA5')#dire)
#plt.ylim(.005,.16)
#plt.legend()
#plt.show()
#fig.savefig('img/era5.'+mon+'.histo.png')

In [ ]:
# plot zonal means
figz = plt.figure(figsize=[7,7])
zax = np.linspace(-1*latmax,latmax,ny)
#plt.plot(1000*zonal_mean_iwp[::-1]+zonal_mean_swp[::-1],zax,'r-o',label='SWP+CIWP')
#plt.plot(1000*zonal_mean_iwp[::-1],zax,'b-o',label='CIWP')
#plt.plot(1000*zonal_mean_swp[::-1],zax,'g-o',label='SWP')
#plt.plot(1000*(zonal_mean_swp[::-1]+zonal_mean_iwp[::-1]),zax,'k-o',label='CIWP+SWP')
plt.plot(zonal_mean_iwp[::-1],zax,'b-o',label='IWP')
#plt.plot(1000*zonal_mean_swp[:],zax,'g-o',label='SWP')
#plt.plot(1000*(zonal_mean_swp[:]+zonal_mean_iwp[:]),zax,'k-o',label='CIWP+SWP')
plt.xlabel(r'$IWP [kg/m^2]$')
plt.ylabel(r'$Latitude [^oN]$')
if aqua ==1: plt.title('Aqua-MODIS 20'+yr+' zonal means')
if aqua ==0: plt.title('Combined-MODIS 20'+yr+' zonal means')
plt.legend()
plt.show()
figz.savefig('img/modis.20'+yr+'.zonal.jpg',dpi=400)

In [ ]:
# plot global mean maps of CIWP/SWP
print(np.size(mean_grid_iwp[mean_grid_iwp != mean_grid_iwp]))
fig1 = plt.figure(figsize=[11,5])
print(info(mean_grid_iwp))
grdmap(mean_grid_iwp,-180,latmax,.001,0.4,1,'MODIS 20'+yr+' IWP mean')#+mon)
#grdmap(mean_grid_iwp,lomin=-180,lamin=70,vmin=.001,vmax=0.4,expflag=1,'ERA5 IWP')
if aqua ==0: fig1.savefig('img/modis-combined.20'+yr+'.iwp_grid.jpg',dpi=500)
if aqua ==1: fig1.savefig('img/modis-aqua.20'+yr+'.iwp_grid.jpg',dpi=500)
#fig2 = plt.figure(figsize=[11,5])
#print(info(mean_grid_swp))
#grdmap(mean_grid_swp,-180,70,.001,0.2,1,'ERA5 20'+yr+' SWP mean')#+mon)
##grdmap(mean_grid_swp,lomin=-180,lamin=70,vmin=.001,vmax=0.4,expflag=1)
#fig2.savefig('img/era5.20'+yr+'.swp_grid.png',dpi=300)